# Documentation lib DataViz

## La classe Data Reader 

Avant propos : Cette classe a pour objectif de servir de lecteur bas niveau pour les différents fichiers Excel de l'application. Elle sert principalement à renvoyer des sous matrices issues des tableaux Excel. Voici son implémentation  

In [ ]:
import openpyxl
from openpyxl.utils import column_index_from_string
import re


# cette classe est tres utile pour lire une matrice delimitee par un mot d entree premiere ligne 
# et pour un mot de sortie pour la derniere ligne 
# on pourra peut etre rajouter une logique permettant de gerer l entete du fichier
# mais ce n est pas a l ordre du jour 
class DataReader1:
    def __init__(self, file_path , start_row_key_word , end_row_keyword  ):
        self.file_path = file_path
        self.matrice = []

        self.main_matrice_start_row = 0
        self.main_matrice_end_row = 0
        self.start_row_key_word =  start_row_key_word 
        self.end_row_keyword  =  end_row_keyword  
        self.workbook =  openpyxl.load_workbook(self.file_path)
        self.determine_start_Row_and_end_Row( start_row_key_word , end_row_keyword   )
        # self.start_main_row = start_main_row


    # permet de déterminer le debut d une ligne et la fin d une ligne du fichier excel  
    def determine_start_Row_and_end_Row(self, key_word_start_row, key_word_end_row, column_search=1):
        try:
            # print(" column search : " + str(column_search))
            sheet = self.workbook.active
            #print("Sheet")
            #print(str(sheet))
            start_row = 1
            end_row = 1
            found_start_row = False
            found_end_row = False

            while start_row < sheet.max_row and not found_start_row  :
                cell_value = sheet.cell(row=start_row, column=column_search).value
                if re.search(str(key_word_start_row), str(cell_value), re.IGNORECASE):
                    found_start_row = True
                else:
                    start_row += 1

            end_row =  start_row + 1  

            while end_row < sheet.max_row and not found_end_row :
                cell_value = sheet.cell(row=end_row, column=column_search).value
                if re.search(str(key_word_end_row), str(cell_value), re.IGNORECASE):
                    found_end_row = True
                else:
                    end_row += 1

            self.main_matrice_start_row = start_row
            self.main_matrice_end_row = end_row - 1

            print("Numero de la ligne où ça commence : ", start_row)
            print("Numero de la ligne où ça finit : ", end_row)

        except Exception as e:
            print("Problème lors de la détermination de la première et dernière ligne de la matrice :", e)

    def return_main_first_row_header(self ):
        # Fetch the row from the worksheet
        row = self.workbook.active [self.main_matrice_start_row]
        # Convert each cell in the row to a string and return as a list
        return [str(cell.value) if cell.value is not None else "" for cell in row]

    # permet de charger une colonne en particulier 
    def load_column(self, column_number, start_row , limit_row):
        try:
            # self.workbook = openpyxl.load_self.workbook(self.file_path)
            sheet = self.workbook.active
            column_data = []
            for row in range(start_row,  limit_row - start_row):
                cell_value = sheet.cell(row=row, column=column_number).value
                column_data.append(cell_value)
            return column_data
        except FileNotFoundError:
            print("Le fichier spécifié est introuvable.")
            return None
        except Exception as e:
            print("Une erreur s'est produite lors du chargement de la colonne :", e)
            return None

    # permet de charger toute la colonne par le nom 
    def load_all_columns_by_name(self, row_number  , end_row_number  , pattern):
        try:
            all_columns_data = []
            sheet = self.workbook.active
            # print(" VALEUR DE SHEET " + str( sheet.cell( row=self.main_matrice_start_row , column=2).value  ) )
            for col in range(1, sheet.max_column + 1):
                cell_value = sheet.cell(row=row_number, column=col).value
                if cell_value and re.search(pattern, str(cell_value), re.IGNORECASE):
                    column_data = self.load_column(col, row_number , end_row_number)
                    #print(column_data)
                    all_columns_data.append(column_data)

            # Affichage de la matrice ligne par ligne
            #print(" affichage ligne par ligne de  " + str( all_columns_data ))
            # for row_data in zip(*all_columns_data):
                #print(row_data) 
            return all_columns_data
        except FileNotFoundError:
            #print("Le fichier spécifié est introuvable.")
            return None
        except Exception as e:
            print("Une erreur s'est produite lors du chargement de toutes les colonnes par nom :", e)
            return None 

    # permet de charger toutes les colonnes a partir d un pattern  (par ex type obj_12 pour avoir toutes les infos d une UE ) 
    def load_all_columns_by_name_from_the_current_line(self,  pattern):
        return self.load_all_columns_by_name( self.main_matrice_start_row , self.main_matrice_end_row , pattern )
    
    def load_matrix_from_current_line_name( self , listePattern  ):
        matrix = [ ]
        for pattern in listePattern: 
            for subColonne in  self.load_all_columns_by_name_from_the_current_line(pattern):
                matrix.append(subColonne)

        # Affichage de la matrice ligne par ligne
        # for row_data in zip(*matrix):
            #print(row_data) 

    # obtenir le num de ligne et de colonne à partir du titre de la colonne et de la valeur recherchee 
    def getRowIndexValue( self ,  title_column , value_that_we_are_looking_for  ):
        indexRow = self.main_matrice_start_row
        column = self.load_all_columns_by_name(self.main_matrice_start_row , self.main_matrice_end_row , title_column)
        for items in column:
            if items and  re.search(value_that_we_are_looking_for   , str(items), re.IGNORECASE):
                break 
            else:
                indexRow += 1 
        indexRow += 1 
        #print(" Le numero de la ligne de la valeur " + str(indexRow))
        return indexRow

    # renvoit un index en lettre de la colonne courante 
    def getColIndexLetterValue(self , titleColumn):
        try:
            sheet = self.workbook.active
            ligne = sheet[self.main_matrice_start_row]
            for items in ligne:
                # print(str(items.value) )
                if items and re.search(titleColumn , str(items.value), re.IGNORECASE):
                    return items.column_letter
            return ""
        except FileNotFoundError:
            print("Le fichier spécifié est introuvable.")
            return None
        except Exception as e:
            print("Une erreur s'est produite lors du chargement de toutes les colonnes par nom :", e)
            return None 

    # 
    def get_corresponding_value_of_another_column( self ,  title_column_1 , value_column_1 ,  title_column_2  ) :
        ligne_referent =  self.getRowIndexValue( title_column_1 , value_column_1  )
        lettre_colonne_cherchee = self.getColIndexLetterValue( title_column_2)
        sheet = self.workbook.active
        # Accéder à la cellule et lire sa valeur
        #print(" VALEUR RECHERCHEE " + str( lettre_colonne_cherchee   ) + str(ligne_referent) )
        #print( sheet[lettre_colonne_cherchee  + str(ligne_referent)].value)
        return sheet[lettre_colonne_cherchee  + str(ligne_referent)].value

    def get_column_letter(self , column_index):
        """ Convertit un index de colonne numérique en lettre de colonne. """
        string = ""
        while column_index > 0:
            column_index, remainder = divmod(column_index - 1, 26)
            string = chr(65 + remainder) + string
        return string
     
    def loading_full_line( self , row_number  ):
        sheet = self.workbook.active 
        row = sheet[ row_number  ]
        row_values = [ cell.value for cell in row ]
        return row_values 
    
    def loading_header_matrice_and_one_tuple( self , title_column_referent , value_referent ):
        header_line = self.loading_full_line( self.main_matrice_start_row )
        row_valued_index = self.getRowIndexValue( title_column_referent ,  value_referent  ) 
        row_valued = self.loading_full_line( row_valued_index  )
        dictionary_returned = { key : value  for key , value in zip( header_line  ,  row_valued  )}

        # for key , value in dictionary_returned.items():
            #print(f"Key: {key}, Value: {value}")

        return  dictionary_returned 

Bien maintenant passons en revue chaque fonction pour voir leur utilité 

## Le constructeur de la classe DataReader 

In [ ]:
# cette classe est tres utile pour lire une matrice delimitee par un mot d entree premiere ligne 
# et pour un mot de sortie pour la derniere ligne 
# on pourra peut etre rajouter une logique permettant de gerer l entete du fichier
# mais ce n est pas a l ordre du jour 
class DataReader1:
    def __init__(self, file_path , start_row_key_word , end_row_keyword  ):
        self.file_path = file_path
        self.matrice = []

        self.main_matrice_start_row = 0
        self.main_matrice_end_row = 0
        self.start_row_key_word =  start_row_key_word 
        self.end_row_keyword  =  end_row_keyword  
        self.workbook =  openpyxl.load_workbook(self.file_path)
        self.determine_start_Row_and_end_Row( start_row_key_word , end_row_keyword   )
        # self.start_main_row = start_main_row


## La méthode determine_start_Row_and_end_Row

In [ ]:

    # permet de déterminer le debut d une ligne et la fin d une ligne du fichier excel  
    def determine_start_Row_and_end_Row(self, key_word_start_row, key_word_end_row, column_search=1):
        try:
            # print(" column search : " + str(column_search))
            sheet = self.workbook.active
            #print("Sheet")
            #print(str(sheet))
            start_row = 1
            end_row = 1
            found_start_row = False
            found_end_row = False

            while start_row < sheet.max_row and not found_start_row  :
                cell_value = sheet.cell(row=start_row, column=column_search).value
                if re.search(str(key_word_start_row), str(cell_value), re.IGNORECASE):
                    found_start_row = True
                else:
                    start_row += 1

            end_row =  start_row + 1  

            while end_row < sheet.max_row and not found_end_row :
                cell_value = sheet.cell(row=end_row, column=column_search).value
                if re.search(str(key_word_end_row), str(cell_value), re.IGNORECASE):
                    found_end_row = True
                else:
                    end_row += 1

            self.main_matrice_start_row = start_row
            self.main_matrice_end_row = end_row - 1

            print("Numero de la ligne où ça commence : ", start_row)
            print("Numero de la ligne où ça finit : ", end_row)

        except Exception as e:
            print("Problème lors de la détermination de la première et dernière ligne de la matrice :", e)


Cette méthode  pour but de déterminer le numéro de la première et de la dernière ligne de la matrice. Pour ce faire, la méthode prend en paramètre le start_keyword et le end_keyword. Les deux variables start_row et end_row prendrons comme valeur le numéro de la ligne ou se trouve le start_key_word et même chose pour le end_key_word et le end_row. 

##  load_all_columns_by_name


In [ ]:
    # permet de charger toute la colonne par le nom 
    def load_all_columns_by_name(self, row_number  , end_row_number  , pattern):
        try:
            all_columns_data = []
            sheet = self.workbook.active
            # print(" VALEUR DE SHEET " + str( sheet.cell( row=self.main_matrice_start_row , column=2).value  ) )
            for col in range(1, sheet.max_column + 1):
                cell_value = sheet.cell(row=row_number, column=col).value
                if cell_value and re.search(pattern, str(cell_value), re.IGNORECASE):
                    column_data = self.load_column(col, row_number , end_row_number)
                    #print(column_data)
                    all_columns_data.append(column_data)

            # Affichage de la matrice ligne par ligne
            #print(" affichage ligne par ligne de  " + str( all_columns_data ))
            # for row_data in zip(*all_columns_data):
                #print(row_data) 
            return all_columns_data
        except FileNotFoundError:
            #print("Le fichier spécifié est introuvable.")
            return None
        except Exception as e:
            print("Une erreur s'est produite lors du chargement de toutes les colonnes par nom :", e)
            return None 

Cette fonction prend en paramètre un numéro de ligne de début et de fin et un pattern. Cette fonction va parcourir la ligne row_number à la recherche du pattern, puis renvoit la colonne dont le titre est le pattern. 

## load_all_columns_by_name_from_the_current_line

In [ ]:
    # permet de charger toutes les colonnes a partir d un pattern  (par ex type obj_12 pour avoir toutes les infos d une UE ) 
    def load_all_columns_by_name_from_the_current_line(self,  pattern):
        return self.load_all_columns_by_name( self.main_matrice_start_row , self.main_matrice_end_row , pattern )

Cette fonction ne prend que le pattern en argument et fait la même chose que load_all_columns_by_name mais avec la première ligne de la matrice principale et la dernière ligne de la matrice principale. 

## load_matrix_from_current_line_name

In [ ]:
    
    def load_matrix_from_current_line_name( self , listePattern  ):
        matrix = [ ]
        for pattern in listePattern: 
            for subColonne in  self.load_all_columns_by_name_from_the_current_line(pattern):
                matrix.append(subColonne)

        # Affichage de la matrice ligne par ligne
        # for row_data in zip(*matrix):
            #print(row_data) 


In [ ]:
Cette fonction permet de charger TOUTES les colonnes respectant la liste de Pattern donnée 

##  getRowIndexValue

In [ ]:
    # obtenir le num de ligne et de colonne à partir du titre de la colonne et de la valeur recherchee 
    def getRowIndexValue( self ,  title_column , value_that_we_are_looking_for  ):
        indexRow = self.main_matrice_start_row
        column = self.load_all_columns_by_name(self.main_matrice_start_row , self.main_matrice_end_row , title_column)
        for items in column:
            if items and  re.search(value_that_we_are_looking_for   , str(items), re.IGNORECASE):
                break 
            else:
                indexRow += 1 
        indexRow += 1 
        #print(" Le numero de la ligne de la valeur " + str(indexRow))
        return indexRow

Permet de prendre en paramètre le titre d'une colonne et un valeur que l'on recherche. Puis le tout renvoit le numéro de la ligne de la valeur en question. 

## getColIndexLetterValue

In [ ]:
    # renvoit un index en lettre de la colonne courante 
    def getColIndexLetterValue(self , titleColumn):
        try:
            sheet = self.workbook.active
            ligne = sheet[self.main_matrice_start_row]
            for items in ligne:
                # print(str(items.value) )
                if items and re.search(titleColumn , str(items.value), re.IGNORECASE):
                    return items.column_letter
            return ""
        except FileNotFoundError:
            print("Le fichier spécifié est introuvable.")
            return None
        except Exception as e:
            print("Une erreur s'est produite lors du chargement de toutes les colonnes par nom :", e)
            return None 



Permet d obtenir le numéro de la colonne d 'un titre de colonne 

## get_corresponding_value_of_another_column

In [ ]:
    def get_corresponding_value_of_another_column( self ,  title_column_1 , value_column_1 ,  title_column_2  ) :
        ligne_referent =  self.getRowIndexValue( title_column_1 , value_column_1  )
        lettre_colonne_cherchee = self.getColIndexLetterValue( title_column_2)
        sheet = self.workbook.active
        # Accéder à la cellule et lire sa valeur
        #print(" VALEUR RECHERCHEE " + str( lettre_colonne_cherchee   ) + str(ligne_referent) )
        #print( sheet[lettre_colonne_cherchee  + str(ligne_referent)].value)
        return sheet[lettre_colonne_cherchee  + str(ligne_referent)].value


Permet d'obtenir à partir d'une colonne et d'une valeur située dans cette colonne, la même valeur sur une même ligne d'une colonne différente. 

## loading_full_line

In [ ]:
    def loading_full_line( self , row_number  ):
        sheet = self.workbook.active 
        row = sheet[ row_number  ]
        row_values = [ cell.value for cell in row ]
        return row_values 

## loading_header_matrice_and_one_tuple

In [ ]:
    def loading_header_matrice_and_one_tuple( self , title_column_referent , value_referent ):
        header_line = self.loading_full_line( self.main_matrice_start_row )
        row_valued_index = self.getRowIndexValue( title_column_referent ,  value_referent  ) 
        row_valued = self.loading_full_line( row_valued_index  )
        dictionary_returned = { key : value  for key , value in zip( header_line  ,  row_valued  )}

        # for key , value in dictionary_returned.items():
            #print(f"Key: {key}, Value: {value}")

        return  dictionary_returned 